# Gordon

In [12]:
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\markf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
df_gordon = pd.read_json("../data/experiment_gordon.jsonl", lines=True)
df_gordon

,sentence,condition_sentence,condition_distractor,condition_question,distractor_value,subject_value,question_center,question
0,The banker that the barber praised climbed the...,subject_extracted,definite_description,matrix_verb,The barber,The banker,subject,The banker climbed the mountain.
1,The banker that the barber praised climbed the...,subject_extracted,definite_description,matrix_verb,The barber,The banker,distractor,The barber climbed the mountain.
2,The banker that you praised climbed the mounta...,subject_extracted,indexical_pronoun,matrix_verb,You,The banker,subject,The banker climbed the mountain.
3,The banker that you praised climbed the mounta...,subject_extracted,indexical_pronoun,matrix_verb,You,The banker,distractor,You climbed the mountain.
4,The banker that Sue praised climbed the mounta...,subject_extracted,name,matrix_verb,Sue,The banker,subject,The banker climbed the mountain.
...,...,...,...,...,...,...,...,...
571,The gardener that envied the homeowner was ver...,object_extracted,definite_description,embedded_verb,The homeowner,The gardener,distractor,The homeowner envied the gardener.
572,The gardener that envied you was very friendly.,object_extracted,indexical_pronoun,embedded_verb,You,The gardener,subject,The gardener envied you.
573,The gardener that envied you was very friendly.,object_extracted,indexical_pronoun,embedded_verb,You,The gardener,distractor,You envied the gardener.
574,The gardener that envied Liz was very friendly.,object_extracted,name,embedded_verb,Liz,The gardener,subject,The gardener envied Liz.


In [14]:
# remove the rows with wrong questions
rows_to_drop = []
rows_to_drop.extend(df_gordon[
    (df_gordon['condition_sentence'] == 'object_extracted') &
    (df_gordon['question_center'] == 'distractor')
].index)

rows_to_drop.extend(df_gordon[
    (df_gordon['condition_sentence'] == 'subject_extracted') &
    (df_gordon['condition_question'] == 'matrix_verb') &
    (df_gordon['question_center'] == 'distractor')
].index)

rows_to_drop.extend(df_gordon[
    (df_gordon['condition_sentence'] == 'subject_extracted') &
    (df_gordon['condition_question'] == 'embedded_verb') &
    (df_gordon['question_center'] == 'subject')
].index)

df_gordon = df_gordon.drop(rows_to_drop)

df_gordon

,sentence,condition_sentence,condition_distractor,condition_question,distractor_value,subject_value,question_center,question
0,The banker that the barber praised climbed the...,subject_extracted,definite_description,matrix_verb,The barber,The banker,subject,The banker climbed the mountain.
2,The banker that you praised climbed the mounta...,subject_extracted,indexical_pronoun,matrix_verb,You,The banker,subject,The banker climbed the mountain.
4,The banker that Sue praised climbed the mounta...,subject_extracted,name,matrix_verb,Sue,The banker,subject,The banker climbed the mountain.
7,The banker that the barber praised climbed the...,subject_extracted,definite_description,embedded_verb,The barber,The banker,distractor,The barber praised the banker.
9,The banker that you praised climbed the mounta...,subject_extracted,indexical_pronoun,embedded_verb,You,The banker,distractor,You praised the banker.
...,...,...,...,...,...,...,...,...
566,The gardener that envied you was very friendly.,object_extracted,indexical_pronoun,matrix_verb,You,The gardener,subject,The gardener was very friendly.
568,The gardener that envied Liz was very friendly.,object_extracted,name,matrix_verb,Liz,The gardener,subject,The gardener was very friendly.
570,The gardener that envied the homeowner was ver...,object_extracted,definite_description,embedded_verb,The homeowner,The gardener,subject,The gardener envied the homeowner.
572,The gardener that envied you was very friendly.,object_extracted,indexical_pronoun,embedded_verb,You,The gardener,subject,The gardener envied you.


In [15]:
# fix some lowercase problems
df_gordon.loc[df_gordon.question.str.contains("The"), "question"] = df_gordon.question.str.replace("The", "the")
df_gordon.loc[df_gordon.question.str.contains("You"), "question"] = df_gordon.question.str.replace("You", "you")
df_gordon['question'] = df_gordon['question'].apply(lambda x: 'Therefore, ' + x)

# fix questions with # instead of the subject
def replace_question(row):
    lower_case_words = ['the', 'you']
    if row['condition_sentence'] == 'subject_extracted':
        value = row['subject_value']
        if any(word in value.lower() for word in lower_case_words):
            value = value.lower()
        return row['question'].replace('#', value)
    elif row['condition_sentence'] == 'object_extracted':
        value = row['distractor_value']
        if any(word in value.lower() for word in lower_case_words):
            value = value.lower()
        return row['question'].replace('#', value)
    else:
        return row['question']

df_gordon['question'] = df_gordon.apply(replace_question, axis=1)

df_gordon

,sentence,condition_sentence,condition_distractor,condition_question,distractor_value,subject_value,question_center,question
0,The banker that the barber praised climbed the...,subject_extracted,definite_description,matrix_verb,The barber,The banker,subject,"Therefore, the banker climbed the mountain."
2,The banker that you praised climbed the mounta...,subject_extracted,indexical_pronoun,matrix_verb,You,The banker,subject,"Therefore, the banker climbed the mountain."
4,The banker that Sue praised climbed the mounta...,subject_extracted,name,matrix_verb,Sue,The banker,subject,"Therefore, the banker climbed the mountain."
7,The banker that the barber praised climbed the...,subject_extracted,definite_description,embedded_verb,The barber,The banker,distractor,"Therefore, the barber praised the banker."
9,The banker that you praised climbed the mounta...,subject_extracted,indexical_pronoun,embedded_verb,You,The banker,distractor,"Therefore, you praised the banker."
...,...,...,...,...,...,...,...,...
566,The gardener that envied you was very friendly.,object_extracted,indexical_pronoun,matrix_verb,You,The gardener,subject,"Therefore, the gardener was very friendly."
568,The gardener that envied Liz was very friendly.,object_extracted,name,matrix_verb,Liz,The gardener,subject,"Therefore, the gardener was very friendly."
570,The gardener that envied the homeowner was ver...,object_extracted,definite_description,embedded_verb,The homeowner,The gardener,subject,"Therefore, the gardener envied the homeowner."
572,The gardener that envied you was very friendly.,object_extracted,indexical_pronoun,embedded_verb,You,The gardener,subject,"Therefore, the gardener envied you."


In [16]:
# create 'label' column

df_gordon['label'] = df_gordon.apply(lambda row: row['subject_value'].split()[-1] if row['question_center'] == 'subject' else row['distractor_value'].split()[-1], axis=1)

# 'you' label should be in lowercase
df_gordon['label'] = df_gordon['label'].str.replace('You', 'you', case=False)

In [17]:
# create 'distractor' column

df_gordon['distractor'] = df_gordon.apply(lambda row: row['subject_value'].split()[-1] if row['question_center'] == 'distractor' else row['distractor_value'].split()[-1], axis=1)

# 'you' label should be in lowercase
df_gordon['distractor'] = df_gordon['distractor'].str.replace('You', 'you', case=False)

In [18]:
# mask the label

def mask_prompt(row):
    mask = '[MASK]'
    label = row['label']
    if row['question_center'] == 'subject':
        if len(label.split()) > 1:
            label = label.split()[1]
        new_row = row['question'].replace(label, mask)
        new_row = new_row.replace(label.lower(), mask)
        return new_row
    elif row['question_center'] == 'distractor':
        if len(label.split()) > 1:
            label = label.split()[1]
        new_row = row['question'].replace(label, mask)
        new_row = new_row.replace(label.lower(), mask)
        return new_row
    else:
        return row['question']

df_gordon['question'] = df_gordon.apply(mask_prompt, axis=1)
df_gordon

,sentence,condition_sentence,condition_distractor,condition_question,distractor_value,subject_value,question_center,question,label,distractor
0,The banker that the barber praised climbed the...,subject_extracted,definite_description,matrix_verb,The barber,The banker,subject,"Therefore, the [MASK] climbed the mountain.",banker,barber
2,The banker that you praised climbed the mounta...,subject_extracted,indexical_pronoun,matrix_verb,You,The banker,subject,"Therefore, the [MASK] climbed the mountain.",banker,you
4,The banker that Sue praised climbed the mounta...,subject_extracted,name,matrix_verb,Sue,The banker,subject,"Therefore, the [MASK] climbed the mountain.",banker,Sue
7,The banker that the barber praised climbed the...,subject_extracted,definite_description,embedded_verb,The barber,The banker,distractor,"Therefore, the [MASK] praised the banker.",barber,banker
9,The banker that you praised climbed the mounta...,subject_extracted,indexical_pronoun,embedded_verb,You,The banker,distractor,"Therefore, [MASK] praised the banker.",you,banker
...,...,...,...,...,...,...,...,...,...,...
566,The gardener that envied you was very friendly.,object_extracted,indexical_pronoun,matrix_verb,You,The gardener,subject,"Therefore, the [MASK] was very friendly.",gardener,you
568,The gardener that envied Liz was very friendly.,object_extracted,name,matrix_verb,Liz,The gardener,subject,"Therefore, the [MASK] was very friendly.",gardener,Liz
570,The gardener that envied the homeowner was ver...,object_extracted,definite_description,embedded_verb,The homeowner,The gardener,subject,"Therefore, the [MASK] envied the homeowner.",gardener,homeowner
572,The gardener that envied you was very friendly.,object_extracted,indexical_pronoun,embedded_verb,You,The gardener,subject,"Therefore, the [MASK] envied you.",gardener,you


In [19]:
# create 'prompt' column

df_gordon["prompt"] = df_gordon["sentence"] + " " + df_gordon["question"]
df_gordon

,sentence,condition_sentence,condition_distractor,condition_question,distractor_value,subject_value,question_center,question,label,distractor,prompt
0,The banker that the barber praised climbed the...,subject_extracted,definite_description,matrix_verb,The barber,The banker,subject,"Therefore, the [MASK] climbed the mountain.",banker,barber,The banker that the barber praised climbed the...
2,The banker that you praised climbed the mounta...,subject_extracted,indexical_pronoun,matrix_verb,You,The banker,subject,"Therefore, the [MASK] climbed the mountain.",banker,you,The banker that you praised climbed the mounta...
4,The banker that Sue praised climbed the mounta...,subject_extracted,name,matrix_verb,Sue,The banker,subject,"Therefore, the [MASK] climbed the mountain.",banker,Sue,The banker that Sue praised climbed the mounta...
7,The banker that the barber praised climbed the...,subject_extracted,definite_description,embedded_verb,The barber,The banker,distractor,"Therefore, the [MASK] praised the banker.",barber,banker,The banker that the barber praised climbed the...
9,The banker that you praised climbed the mounta...,subject_extracted,indexical_pronoun,embedded_verb,You,The banker,distractor,"Therefore, [MASK] praised the banker.",you,banker,The banker that you praised climbed the mounta...
...,...,...,...,...,...,...,...,...,...,...,...
566,The gardener that envied you was very friendly.,object_extracted,indexical_pronoun,matrix_verb,You,The gardener,subject,"Therefore, the [MASK] was very friendly.",gardener,you,The gardener that envied you was very friendly...
568,The gardener that envied Liz was very friendly.,object_extracted,name,matrix_verb,Liz,The gardener,subject,"Therefore, the [MASK] was very friendly.",gardener,Liz,The gardener that envied Liz was very friendly...
570,The gardener that envied the homeowner was ver...,object_extracted,definite_description,embedded_verb,The homeowner,The gardener,subject,"Therefore, the [MASK] envied the homeowner.",gardener,homeowner,The gardener that envied the homeowner was ver...
572,The gardener that envied you was very friendly.,object_extracted,indexical_pronoun,embedded_verb,You,The gardener,subject,"Therefore, the [MASK] envied you.",gardener,you,The gardener that envied you was very friendly...


In [20]:
# drop columns

# df_gordon.drop(["condition_sentence", "condition_distractor", "condition_question", "distractor_value",  "subject_value", "question_center"], axis=1, inplace=True)
df_gordon.drop(["sentence", "question"], axis=1, inplace=True)

In [21]:
# change the order of the columns

df_gordon = df_gordon[["prompt", "label", "distractor", "condition_sentence", "condition_distractor", "condition_question", "distractor_value",  "subject_value", "question_center"]]

In [22]:
df_gordon

,prompt,label,distractor,condition_sentence,condition_distractor,condition_question,distractor_value,subject_value,question_center
0,The banker that the barber praised climbed the...,banker,barber,subject_extracted,definite_description,matrix_verb,The barber,The banker,subject
2,The banker that you praised climbed the mounta...,banker,you,subject_extracted,indexical_pronoun,matrix_verb,You,The banker,subject
4,The banker that Sue praised climbed the mounta...,banker,Sue,subject_extracted,name,matrix_verb,Sue,The banker,subject
7,The banker that the barber praised climbed the...,barber,banker,subject_extracted,definite_description,embedded_verb,The barber,The banker,distractor
9,The banker that you praised climbed the mounta...,you,banker,subject_extracted,indexical_pronoun,embedded_verb,You,The banker,distractor
...,...,...,...,...,...,...,...,...,...
566,The gardener that envied you was very friendly...,gardener,you,object_extracted,indexical_pronoun,matrix_verb,You,The gardener,subject
568,The gardener that envied Liz was very friendly...,gardener,Liz,object_extracted,name,matrix_verb,Liz,The gardener,subject
570,The gardener that envied the homeowner was ver...,gardener,homeowner,object_extracted,definite_description,embedded_verb,The homeowner,The gardener,subject
572,The gardener that envied you was very friendly...,gardener,you,object_extracted,indexical_pronoun,embedded_verb,You,The gardener,subject


In [23]:
df_gordon.to_csv("../data_processed/gordon.csv", index=False)